# BG1 tranlate Dataset Build

## (1) tlk parser

In [1]:
# tlk parser from https://github.com/3zhang/TLK-v1-file-parser-for-Python

# -*- coding: utf-8 -*-

#Author 3Zhang

import struct

b2i=lambda bstr:struct.unpack('i',bstr)[0]
b2h=lambda bstr:struct.unpack('h',bstr)[0]
i2b=lambda i:struct.pack('i',i)
h2b=lambda i:struct.pack('h',i)


#This is the version information of the header. Do not edit.
HEADER_C=b'\x54\x4c\x4b\x20\x56\x31\x20\x20\x00\x00'
COLORRED="\033[01;31m{0}\033[00m"


#a class to store each entry
class entry:
    def __init__(self,ent,n,string=None):
        self.No=n
        self.entype=b2h(ent[:2])
        self.soundinfo=ent[2:18]
        self.offset=b2i(ent[18:22])
        self.size=b2i(ent[22:])
        self.string=string
        
    def __repr__(self):
        return repr((self.No,self.entype,self.soundinfo,self.offset,self.size,self.string.decode('utf-8')))
    
    def __eq__(self, other):
        return (self.No,self.entype,self.soundinfo,self.offset,self.size,self.string) \
            == (other.No,other.entype,other.soundinfo,other.offset,other.size,other.string)


#read a tlk file. return a list of entries
def readialog(filepath):
    with open(filepath,'rb') as file:
        dg=file.read()
    header=dg[:18]
    str_o=b2i(header[14:18])
    strings=dg[str_o:]
    entries=dg[18:str_o]
    entry_l=[entries[i:i+26] for i in range(0,len(entries),26)]
    entry_l2=[entry(ent,i) for i,ent in enumerate(entry_l)]
    for i,ent in enumerate(entry_l2):
        entry_l2[i].string=strings[ent.offset:ent.offset+ent.size]
    return entry_l2
        
#Edit your dialog here. Note that string for each entry needs to be decode.
#Also, after you edit the strings, you need to encode them to binary strings.


#sort a list of entries and refresh its size and offset. You must do this after you finish editing the strings.
def refreshdialog(entryl):
    if sum([not isinstance(ent.string,bytes) for ent in entryl])>0:
        raise TypeError('String must be encoded to bytes!')
    entryl.sort(key=lambda x:x.No)
    if [ent.No for ent in entryl]!=list(range(0,len(entryl))):
        print(COLORRED.format('Warning: List index is not equal to stringref index!'))
    offset=0
    for i,ent in enumerate(entryl):
        size=len(ent.string)
        entryl[i].size=size    
        entryl[i].offset=offset if size>0 else 0
        offset+=size
        

#You must refresh the list of entries before you write them to file.
def writedialog(entryl,filepath):
    length=i2b(len(entryl))
    entries=[]
    for ent in entryl:
        entype=h2b(ent.entype)
        soundinfo=ent.soundinfo
        offset=i2b(ent.offset)
        size=i2b(ent.size)
        entb=entype+soundinfo+offset+size
        entries.append(entb)
    entries=b''.join(entries)
    soffset=i2b(18+len(entries))
    header=HEADER_C+length+soffset
    strings=[ent.string for ent in entryl]
    strings=b''.join(strings)
    dialog=header+entries+strings
    with open(filepath,'wb') as file:
        file.write(dialog)

## (2) BG1 Dialog extract

In [2]:
dial_en = readialog(r"../train_data/BG1(raw)/dialog_en.tlk")
dial_kr = readialog(r"../train_data/BG1(raw)/dialog_ko.tlk")

In [3]:
print(len(dial_kr))

dial_kr[:5]

34000


[(0, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0, 9, '<NO TEXT>'),
 (1, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 9, 86, '왜 여기서 나를 방해하지? 그대는 예의라는 것을 모르나? 나가게!'),
 (2, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 95, 437, '그래, <CHARNAME>. 그대는 그대 아버지 고라이온의 고향을 더럽히고 수세기에 걸쳐 유지해온 평화에 몰락을 불러와 그 이름에 먹칠을 했네. 그대와 그대의 동료들은 경멸을 받아 마땅하며, 그 죄는 가장 가혹한 형벌로 다스려질 것이네. 나는 그대를 브루노스 코스탁, 그리고 리엘타 안체브를 살해한 혐의로 정식으로 기소하는 바이네.'),
 (3, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 532, 346, '내가 왜 그대를 기소했겠나? 그대가 살인 현장에서 도망치던 모습을 누군가 보았단 말이네. 코베라스가 그대의 소지품에서 섀도우 씨프 암살자임을 증명하는 반지와 앰에서 주조된 금화를 발견했지. 이 정도 증거라면 그대를 기소하기에 충분하다고 생각하네만.'),
 (4, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 878, 101, '그대가 피살 현장에서 도주하는 것이 코베라스라는 방문객에게 목격되었네.')]

In [4]:
print(len(dial_kr))

dial_en[:5]

34000


[(0, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 0, 9, '<NO TEXT>'),
 (1, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 9, 63, 'Why hast thou disturbed me here? Hast thou no manners? Get out!'),
 (2, 5, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 72, 307, "So, <CHARNAME>, you have sullied your father's name by defiling his home and bringing ruin to a peace that has lasted for centuries. I spit on you and all of your friends. Your transgressions will be punished in the most severe form. I formally accuse you of the murders of Brunos Costak and Rieltar Anchev."),
 (3, 1, b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', 379, 228, 'Why have I accused you? You were seen fleeing the murder scene. Koveras found the identifying ring of a Shadow Thief assassin in your personal effects, and gold minted in Amn. I feel that it is strong enough proof to accuse you.'),
 (4, 1, b'\x00\x00\x00\x00\x00\x0

In [5]:
dial_en[5].string

b"From everything we have seen and found, it is not hard to figure what your purpose was. From what I have been told, you have been trying to place all the misfortune of the Sword Coast squarely on the shoulders of the Iron Throne. I assume you and your friends are assassins in the employ of Amn. Perhaps you were sent up north to create discord in the region before an Amnian invasion. It matters little; you and your friends are to be sent to Baldur's Gate where an appropriate punishment will be administered."

In [6]:
print(dial_en[24000].string.decode('utf-8'))
print(dial_kr[24000].string.decode('utf-8'))

Andris's Journal
안드리스의 일지


## (2) Preprocess & Make DataFrame

In [7]:
import pandas as pd
import numpy as np

### 1) 해석이 안되어서 겹치는 문장 제거

In [8]:
english_list = []
korean_list = []

for i in range(len(dial_en)):
    english_list.append(dial_en[i].string.decode('utf-8'))
    korean_list.append(dial_kr[i].string.decode('utf-8'))


In [9]:
print(len(korean_list))
print(len(english_list))

double_list =[]
for i, (elem1, elem2) in enumerate(zip(english_list, korean_list)):
    if not elem1 == elem2:
        continue
    double_list.append(i)

print("겹치는 문장개수:", len(double_list))

34000
34000
겹치는 문장개수: 2742


In [10]:
print(len(double_list))
print(len(korean_list))
print(len(english_list))

clean_english_list = []
clean_korean_list = []

for elem1, elem2 in zip(english_list, korean_list):
    if elem1 is not None and elem2 is not None:
        if elem1 != elem2:
            clean_english_list.append(elem1)
            clean_korean_list.append(elem2)
        
print("Done")
print(len(clean_english_list))
print(len(clean_korean_list))

2742
34000
34000
Done
31258
31258


In [11]:
df_dialog_BG1 = pd.DataFrame(columns = ["english", "korean"])

df_dialog_BG1["english"] = clean_english_list
df_dialog_BG1["korean"] = clean_korean_list

In [12]:
df_dialog_BG1.head(10)

,english,korean
0,Why hast thou disturbed me here? Hast thou no ...,왜 여기서 나를 방해하지? 그대는 예의라는 것을 모르나? 나가게!
1,"So, <CHARNAME>, you have sullied your father's...","그래, <CHARNAME>. 그대는 그대 아버지 고라이온의 고향을 더럽히고 수세기에..."
2,Why have I accused you? You were seen fleeing ...,내가 왜 그대를 기소했겠나? 그대가 살인 현장에서 도망치던 모습을 누군가 보았단 말...
3,You were seen fleeing the site of the murder b...,그대가 피살 현장에서 도주하는 것이 코베라스라는 방문객에게 목격되었네.
4,"From everything we have seen and found, it is ...",우리가 지금까지 보고 수집한 사실로부터 그대의 목적을 알아차리는 것은 어려운 일이 ...
5,But I have done nothing wrong! Why have you ac...,하지만 난 아무런 잘못도 하지 않았소! 도대체 왜 그런 일로 날 기소하는 거요?
6,"Fine. Do your worst, you old coot.","좋아, 뭐든지 할 테면 해보시지, 이 늙은 멍청아."
7,But I have done nothing wrong! Why have you ac...,하지만 난 아무런 잘못도 하지 않았소! 도대체 왜 그런 일로 날 기소하는 거요?
8,I am pleased to see thee again. Hast thou come...,다시 만나게 되어 반갑네. 그대 아버지의 유품을 가지러 왔는가? 그는 여러 물건을 ...
9,"Surrender to the guard, young one. Thou woulds...","경비병에게 항복하게, 젊은이. 상황을 악화시킬 뿐이야."


In [13]:
df_dialog_BG1.tail(10)

,english,korean
31248,Aauughh! Dynaheir has been slain! For what pur...,아아악! 다이나헤어가 살해되었다니! 이제 난 무얼 해야 하지? 불명예를 안고 고향으...
31249,"A witch, you say? Oh, right. She's dead.","마녀라고 했소? 오, 그렇지. 그녀는 죽었소."
31250,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
31251,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
31252,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
31253,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."
31254,Marek's Poison,마렉의 독
31255,Find Familiar\n(Conjuration/Summoning)\n\nLeve...,Find Familiar\n(Conjuration/Summoning)\nLevel:...
31256,Shapeshift: Sword Spider\n\nStrength: 16\nDext...,Shapeshift: Sword Spider\nStrength: 16\nDexter...
31257,Psionic Blast\nA mind flayer's preferred metho...,Psionic Blast\nA mind flayer's preferred metho...


### 2) 필요없는 문자열 제거

In [14]:
df_dialog_BG1.drop(df_dialog_BG1.index[31255:31258], axis=0, inplace=True)

In [15]:
df_dialog_BG1.tail()

,english,korean
31250,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
31251,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
31252,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
31253,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."
31254,Marek's Poison,마렉의 독


In [16]:
df_dialog_BG1

,english,korean
0,Why hast thou disturbed me here? Hast thou no ...,왜 여기서 나를 방해하지? 그대는 예의라는 것을 모르나? 나가게!
1,"So, <CHARNAME>, you have sullied your father's...","그래, <CHARNAME>. 그대는 그대 아버지 고라이온의 고향을 더럽히고 수세기에..."
2,Why have I accused you? You were seen fleeing ...,내가 왜 그대를 기소했겠나? 그대가 살인 현장에서 도망치던 모습을 누군가 보았단 말...
3,You were seen fleeing the site of the murder b...,그대가 피살 현장에서 도주하는 것이 코베라스라는 방문객에게 목격되었네.
4,"From everything we have seen and found, it is ...",우리가 지금까지 보고 수집한 사실로부터 그대의 목적을 알아차리는 것은 어려운 일이 ...
...,...,...
31250,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
31251,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
31252,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
31253,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


### 3) 결측치 제거

In [17]:
short_string_indexes = df_dialog_BG1.index[df_dialog_BG1.applymap(lambda x: len(x) == 0).any(axis=1)]

print(short_string_indexes)

Index([29749], dtype='int64')


In [18]:
df_dialog_BG1.iloc[29749]

english     
korean      
Name: 29749, dtype: object

In [19]:
df_dialog_BG1.drop(short_string_indexes, axis=0, inplace=True)

In [20]:
df_dialog_BG1

,english,korean
0,Why hast thou disturbed me here? Hast thou no ...,왜 여기서 나를 방해하지? 그대는 예의라는 것을 모르나? 나가게!
1,"So, <CHARNAME>, you have sullied your father's...","그래, <CHARNAME>. 그대는 그대 아버지 고라이온의 고향을 더럽히고 수세기에..."
2,Why have I accused you? You were seen fleeing ...,내가 왜 그대를 기소했겠나? 그대가 살인 현장에서 도망치던 모습을 누군가 보았단 말...
3,You were seen fleeing the site of the murder b...,그대가 피살 현장에서 도주하는 것이 코베라스라는 방문객에게 목격되었네.
4,"From everything we have seen and found, it is ...",우리가 지금까지 보고 수집한 사실로부터 그대의 목적을 알아차리는 것은 어려운 일이 ...
...,...,...
31250,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
31251,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
31252,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
31253,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


## (3) Concat DataFrame

### 1) Load DataFrame

In [21]:
df_dialog_set = pd.read_csv("../train_data/dialog_BG2_POE2.csv", lineterminator='\n')

In [22]:
df_dialog_set

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
117175,117175,A horrid beast wanders the White Void. Put an ...,무시무시한 짐승이 하얀 간극을 거닐고 있습니다. 그 공허한 존재에 종말을 가져다주십...
117176,117176,A collection of tortured souls wanders forgott...,고통받은 영혼들의 무리가 잊혀진 대지를 거닐고 있습니다. 그것을 찾아내 안식을 선사...
117177,117177,Something dreadful stirs within the Black Isle...,"무시무시한 존재가 검은 군도 내에서 움직이며, 에오타스보다 더 끔찍한 위협을 초래하..."
117178,117178,Quest Failed.,퀘스트에 실패했습니다.


### 2) Concat DataFrame

In [23]:
df_dialog = pd.concat([df_dialog_set, df_dialog_BG1])

In [24]:
df_dialog

,num,english,korean
0,0.0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1.0,You played Elminster?,엘민스터를 연기했습니까?
2,2.0,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3.0,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4.0,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
31250,NaN,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
31251,NaN,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
31252,NaN,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
31253,NaN,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


In [25]:
df_dialog.reset_index(inplace=True)

In [26]:
df_dialog.drop(["num"], axis=1, inplace=True)
df_dialog.drop(["index"], axis=1, inplace=True)

In [27]:
df_dialog

,english,korean
0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,You played Elminster?,엘민스터를 연기했습니까?
2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...
148429,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
148430,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
148431,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
148432,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


In [28]:
df_dialog.reset_index(inplace=True)

In [29]:
df_dialog.rename(columns={"index":"num"}, inplace=True)

In [32]:
df_dialog

,num,english,korean
0,0,"No, I'm sorry, none of them sound familiar.","아니오, 미안하지만, 그것들 가운데 아는 것은 없소."
1,1,You played Elminster?,엘민스터를 연기했습니까?
2,2,"Uh, the yugoloth, was it? Yeah, you stole the ...","어, 유골로스, 맞습니까? 그래요, 내 기억이 맞다면 당신은 그것으로 쇼를 독차지했었소."
3,3,"And, who knows, we were rehearsing for Picocci...","그리고, 누가 압니까, 우리는 피코키오의 ""에테르 속에서의 사흘간""을 연습하고 있었..."
4,4,"Oh, my dark ravens, let us stop our squawking....","오, 이제 불평은 그만둡시다. 만약 당신이 계속 나와 함께 할 생각이라면, 나는 이..."
...,...,...,...
148429,148429,"A witch, you say? Oh, right. I rescued her. We...","마녀라고 했소? 오, 그렇지. 그녀를 구출했소. 우린 헤어졌지."
148430,148430,(She is gone! Rashemaar fools tramping about t...,"(그녀가 가버렸군! 라셰멘 멍청이들이 렐름을 헤매다니, 테이의 눈을 피할 수 있으리..."
148431,148431,A Ring for a Lady\nI have given Fergus a ring ...,숙녀를 위한 반지\n페르거스에게 숙녀에게 사랑을 바칠 반지를 건내주었다. 행운이 따...
148432,148432,Troubles in the Region: Rumors and Happenings\...,"지역 내 문제들\n내쉬켈 광산의 깊숙한 곳을 파헤쳤으므로, 베런 개스트킬에게 돌아가..."


### 3) Save CSV

In [31]:
df_dialog.to_csv(r"../train_data/dialog_BG2_POE2_BG1.csv", index=False, encoding='utf-8')

### 4) Dataset Build

In [ ]:
# datasets_train = DatasetDict({
#     "train": Dataset.from_pandas(df_dialog)
# })

In [ ]:
# datasets_train

### 5) HuggingFace Upload

In [ ]:
# from huggingface_hub import login

# login(token= input("Token: "))

# datasets_train.push_to_hub("aeolian83/DnD_translate_v1.5")